![Image](https://blog.langchain.com/content/images/2024/01/simple_multi_agent_diagram--1-.png)

![Image](https://docs.aws.amazon.com/images/prescriptive-guidance/latest/agentic-ai-patterns/images/workflow-for-prompt-chaining.png)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1200/0%2Afmk9z4CULIH1fd9w.png)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1400/1%2AoWF9Ag_1R7-E9Xv0odsf7A.png)

## Sequential Workflows with LangGraph — Structured Guide

This guide presents a clean, end-to-end explanation of how to build **sequential workflows** using LangGraph, progressing from foundational concepts to a multi-step LLM prompt-chaining example. The structure and logic strictly follow the provided source without altering its intent or scope.

---

## Sequential Workflow Implementation Pattern

A **sequential workflow** in LangGraph is a strictly linear pipeline. Each task executes in order, with no branching or parallelism. Regardless of complexity, every LangGraph project adheres to the same **five-step structural pattern**:

1. **Define State**
   Create a `TypedDict` that represents the shared, persistent memory of the workflow.

2. **Define Nodes**
   Implement Python functions (nodes) that:

   * Accept the current state
   * Perform a unit of logic
   * Return a *partial update* to the state

3. **Build Graph**
   Initialize a `StateGraph`, register nodes, and connect them with directed edges.

4. **Compile**
   Validate and finalize the graph structure.

5. **Execute**
   Invoke the compiled graph with an initial state.

---

## Project Workflow: LLM Prompt Chaining

The most advanced sequential example in the source is a **Blog Generation Workflow**. Instead of a single prompt, the task is decomposed into multiple LLM steps to improve quality, traceability, and state retention.

---

### 1. State Definition

The state object persists across the entire execution. Outputs generated in early nodes (such as the outline) remain accessible to all downstream nodes, including after execution completes.

```python
from typing import TypedDict

class BlogState(TypedDict):
    title: str
    outline: str
    content: str
```

---

### 2. Node Logic

Each node is a pure function that:

* Reads from the shared state
* Calls the LLM
* Returns only the fields it updates

The second node explicitly depends on the output of the first.

```python
def create_outline(state: BlogState):
    prompt = f"Generate a detailed outline for a blog on: {state['title']}"
    response = model.invoke(prompt).content
    return {"outline": response}

def create_blog(state: BlogState):
    prompt = (
        f"Write a blog for '{state['title']}' "
        f"using this outline: {state['outline']}"
    )
    response = model.invoke(prompt).content
    return {"content": response}
```

---

### 3. Graph Construction and Visualization

LangGraph uses two built-in **dummy nodes**—`START` and `END`—to define entry and exit points. These nodes are not user-defined logic; they exist to enforce graph structure.

```python
from langgraph.graph import StateGraph, START, END

builder = StateGraph(BlogState)

builder.add_node("generate_outline", create_outline)
builder.add_node("generate_blog", create_blog)

builder.add_edge(START, "generate_outline")
builder.add_edge("generate_outline", "generate_blog")
builder.add_edge("generate_blog", END)

workflow = builder.compile()
```

In notebook environments (e.g., Jupyter), the compiled graph can be visually rendered to audit execution flow before runtime.

---

### 4. Execution

Invoking the workflow triggers node execution in sequence. The state is incrementally updated and returned in full at the end.

```python
initial_input = {"title": "The Rise of AI in India"}
final_state = workflow.invoke(initial_input)

print(final_state["outline"])
print(final_state["content"])
```

---

## Core Benefits of This Structure

* **State Persistence**
  Intermediate outputs are preserved throughout execution, unlike traditional one-shot chains.

* **Observability**
  The explicit graph structure allows visual inspection and logical validation before execution.

* **Separation of Concerns**
  Each node performs exactly one responsibility, simplifying debugging, iteration, and extension.

---

## Next Logical Extension

A natural extension of this sequential workflow is the addition of a third node—such as an **Evaluator**—that consumes the generated blog content and produces a quality score or critique, further enriching the state without altering the linear execution model.



```python
from langgraph.graph import StateGraph, START, END
from langchain_community.llms import HuggingFaceHub
from typing import TypedDict
import os

# Hugging Face token must be set
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_xxx"
```

---

## Model (Free Hugging Face)

```python
model = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={
        "temperature": 0.3,
        "max_new_tokens": 512
    }
)
```

---

## State Definition (unchanged style)

```python
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    score: str
```

---

## Node 1 — Create Outline (same pattern)

```python
def create_outline(state: BlogState) -> BlogState:
    title = state["title"]

    prompt = f"Generate a detailed outline for a blog on the topic - {title}"
    outline = model.invoke(prompt)

    state["outline"] = outline
    return state
```

---

## Node 2 — Create Blog (same pattern)

```python
def create_blog(state: BlogState) -> BlogState:
    title = state["title"]
    outline = state["outline"]

    prompt = (
        f"Write a detailed blog on the title - {title} "
        f"using the following outline:\n{outline}"
    )

    content = model.invoke(prompt)

    state["content"] = content
    return state
```

---

## ➕ Node 3 — Evaluate Blog (NEW)

```python
def evaluate_blog(state: BlogState) -> BlogState:
    content = state["content"]

    prompt = (
        "Evaluate the following blog on a scale of 1 to 10 "
        "based on clarity, structure, and completeness. "
        "Also give 1–2 lines of feedback.\n\n"
        f"{content}"
    )

    evaluation = model.invoke(prompt)

    state["score"] = evaluation
    return state
```

---

## Graph Construction (same as yours)

```python
graph = StateGraph(BlogState)

graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)
graph.add_node("evaluate_blog", evaluate_blog)

graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", "evaluate_blog")
graph.add_edge("evaluate_blog", END)

workflow = graph.compile()
```

---

## Execution (same style)

```python
initial_state = {
    "title": "Rise of AI in India"
}

final_state = workflow.invoke(initial_state)
```

---

## Outputs

```python
print("OUTLINE:\n", final_state["outline"])
print("\nBLOG:\n", final_state["content"])
print("\nEVALUATION:\n", final_state["score"])
```

---



